In [1]:
! pip install torchtext==0.10.1
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [209]:
from datasets import load_dataset

import torch
from torchtext.legacy import data

In [210]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [211]:
PROJECT_ROOT = F"/content/gdrive/My Drive/nlp_project_task_1/"

In [212]:
SEED = 42
MAX_VOCAB_SIZE = 25_000

In [213]:
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [214]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [215]:
faithdial_dataset = load_dataset("McGill-NLP/FaithDial")

  0%|          | 0/7 [00:00<?, ?it/s]

In [216]:
faithdial_dataset.keys()

dict_keys(['test', 'test_random_split', 'test_topic_split', 'train', 'validation', 'valid_random_split', 'valid_topic_split'])

In [217]:
faithdial_dataset["train"][0]

{'dialog_idx': 0,
 'response': 'Yeah, but once the access to the internet was a rare thing. do you remember?',
 'original_response': "No I could not! I couldn't imagine living when internet access was rare and very few people had it!",
 'history': ['Can you imagine the world without internet access?'],
 'knowledge': 'Internet access was once rare, but has grown rapidly.',
 'BEGIN': ['Hallucination'],
 'VRM': ['Disclosure', 'Ack.']}

In [252]:
def critic_preprocess(dataset):
    """
    Data items transformed into (knowledge, response, is_hallucination)
    """
    new_dataset = []
    for d in dataset:
        # original response
        if d["original_response"] != None:
            new_dataset.append({
                "knowledge": d["knowledge"],
                "response": d["original_response"],
                "hallucination": "yes" if "Hallucination" in d["BEGIN"] else "no",
                "history": " \\ ".join(d["history"])
            })

        # new responses always aren't hallucinations
        new_dataset.append({"knowledge": d["knowledge"], "response": d["response"], "hallucination": "no", "history": "\\".join(d["history"])})
    return new_dataset

In [253]:
import json

def dump_as_json(dataset, filename):
    """
    Takes a list of dicts and dumps it as a json file that torchtext can parse.
    """
    with open(filename, "w") as file:
        for d in dataset:
            file.write(json.dumps(d))
            file.write("\n")


In [254]:
KNOWLEDGE = data.Field(tokenize='spacy', tokenizer_language="en_core_web_sm", include_lengths = True)
RESPONSE = data.Field(tokenize='spacy', tokenizer_language="en_core_web_sm", include_lengths = True)
HISTORY = data.Field(tokenize='spacy', tokenizer_language="en_core_web_sm", include_lengths = True)
LABEL = data.LabelField(dtype=torch.float)

In [255]:
dump_as_json(critic_preprocess(faithdial_dataset["test"]), PROJECT_ROOT + "data/faithdial_dataset_test.json")
dump_as_json(critic_preprocess(faithdial_dataset["train"]), PROJECT_ROOT + "data/faithdial_dataset_train.json")
dump_as_json(critic_preprocess(faithdial_dataset["validation"]), PROJECT_ROOT + "data/faithdial_dataset_validation.json")

In [256]:
fields = {"knowledge": ("k", KNOWLEDGE), "response": ("r", RESPONSE), "hallucination": ("l", LABEL), "history": ("h", HISTORY)}

dataset = data.TabularDataset.splits(path=PROJECT_ROOT + "data",
                                     train="faithdial_dataset_train.json",
                                     validation="faithdial_dataset_validation.json",
                                     test="faithdial_dataset_test.json",
                                     format="json",
                                     fields=fields)


In [257]:
train_data, valid_data, test_data = dataset

In [258]:
train_data[0]

In [259]:
vars(train_data.examples[0])

{'k': ['Internet',
  'access',
  'was',
  'once',
  'rare',
  ',',
  'but',
  'has',
  'grown',
  'rapidly',
  '.'],
 'r': ['No',
  'I',
  'could',
  'not',
  '!',
  'I',
  'could',
  "n't",
  'imagine',
  'living',
  'when',
  'internet',
  'access',
  'was',
  'rare',
  'and',
  'very',
  'few',
  'people',
  'had',
  'it',
  '!'],
 'l': 'yes',
 'h': ['Can',
  'you',
  'imagine',
  'the',
  'world',
  'without',
  'internet',
  'access',
  '?']}

In [265]:
KNOWLEDGE.build_vocab(train_data,
                      max_size=MAX_VOCAB_SIZE,
                      vectors = "fasttext.simple.300d",
                      unk_init = torch.Tensor.normal_)
RESPONSE.build_vocab(train_data,
                     max_size=MAX_VOCAB_SIZE,
                     vectors = "fasttext.simple.300d",
                     unk_init = torch.Tensor.normal_)
HISTORY.build_vocab(train_data,
                     max_size=MAX_VOCAB_SIZE,
                     vectors = "fasttext.simple.300d",
                     unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)


In [266]:
print(f"Unique tokens in KNOWLEDGE vocabulary: {len(KNOWLEDGE.vocab)}")
print(f"Unique tokens in RESPONSE vocabulary: {len(RESPONSE.vocab)}")
print(f"Unique tokens in HISTORY vocabulary: {len(HISTORY.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in KNOWLEDGE vocabulary: 24539
Unique tokens in RESPONSE vocabulary: 25002
Unique tokens in HISTORY vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [267]:
print(KNOWLEDGE.vocab.freqs.most_common(20))
print(RESPONSE.vocab.freqs.most_common(20))
print(HISTORY.vocab.freqs.most_common(20))
print(LABEL.vocab.freqs.most_common(20))

[(',', 61720), ('the', 38383), ('.', 33816), ('and', 28891), ('of', 26828), ('a', 20285), ('in', 19499), ('is', 17156), ("''", 16411), ('to', 12365), ('or', 9842), ('as', 9598), (')', 8496), ('-', 8327), ('(', 8142), ('The', 7870), ('by', 6639), ('with', 5784), ('for', 5683), ('are', 5067)]
[('.', 32770), (',', 31530), ('the', 24162), ('I', 18471), ('a', 14945), ('and', 13738), ('of', 13482), ('is', 12016), ('in', 11802), ('that', 10738), ('to', 10728), ('you', 8747), ('it', 8115), ('?', 7405), ('know', 6604), ('are', 6421), ('!', 6137), ("'s", 6079), ('have', 5205), ('but', 4822)]
[(',', 110829), ('I', 83590), ('.', 74395), ('the', 73633), ('a', 59048), ('you', 51714), ('to', 45654), ('of', 44877), ('is', 42626), ('that', 42319), ('and', 37284), ('know', 37213), ('in', 32563), ('it', 32540), ('?', 29758), ('do', 26824), ('have', 23662), ("'s", 22912), ('are', 20696), ('about', 20335)]
[('no', 20474), ('yes', 13507)]


In [268]:
print(KNOWLEDGE.vocab.itos[:10])
print(RESPONSE.vocab.itos[:10])
print(HISTORY.vocab.itos[:10])
print(LABEL.vocab.itos[:10])

['<unk>', '<pad>', ',', 'the', '.', 'and', 'of', 'a', 'in', 'is']
['<unk>', '<pad>', '.', ',', 'the', 'I', 'a', 'and', 'of', 'is']
['<unk>', '<pad>', ',', 'I', '.', 'the', 'a', 'you', 'to', 'of']
['no', 'yes']


In [269]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    sort_key = lambda x: x.r,
    device = device)

In [270]:
from torch import nn

class LSTM(nn.Module):
    def __init__(self, response_vocab_size, knowledge_vocab_size, history_vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,
                 bidirectional, dropout, response_pad_idx, knowledge_pad_idx, history_pad_idx):

        super().__init__()

        # Initialize Embedding Layer
        self.response_embedding = nn.Embedding(num_embeddings=response_vocab_size,
                                               embedding_dim=embedding_dim,
                                               padding_idx=response_pad_idx)

        self.knowledge_embedding = nn.Embedding(num_embeddings=knowledge_vocab_size,
                                                embedding_dim=embedding_dim,
                                                padding_idx=knowledge_pad_idx)
        
        self.history_embedding = nn.Embedding(num_embeddings=history_vocab_size,
                                              embedding_dim=embedding_dim,
                                              padding_idx=history_pad_idx)

        # Initialize LSTM layer
        self.response_lstm = nn.LSTM(input_size=embedding_dim,
                                     hidden_size=hidden_dim,
                                     num_layers=n_layers,
                                     bidirectional=bidirectional)

        self.knowledge_lstm = nn.LSTM(input_size=embedding_dim,
                                      hidden_size=hidden_dim,
                                      num_layers=n_layers,
                                      bidirectional=bidirectional)
        
        self.history_lstm = nn.LSTM(input_size=embedding_dim,
                                    hidden_size=hidden_dim,
                                    num_layers=n_layers,
                                    bidirectional=bidirectional)

        # Initialize a fully connected layer with Linear transformation
        self.fc = nn.Linear(in_features=3*2*hidden_dim,
                            out_features=output_dim)

        # Initialize Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, response, response_lengths, knowledge, knowledge_lengths, history, history_lengths):
        # Apply embedding layer that matches each word to its vector and apply dropout. Dim [sent_len, batch_size, emb_dim]
        x_r = self.response_embedding(response)
        x_r = self.dropout(x_r)

        x_k = self.knowledge_embedding(knowledge)
        x_k = self.dropout(x_k)

        x_h = self.history_embedding(history)
        x_h = self.dropout(x_h)

        # Run the LSTM along the sentences of length sent_len.
        output_r, (hidden_r, cell_r) = self.response_lstm(x_r)
        output_k, (hidden_k, cell_k) = self.knowledge_lstm(x_k)
        output_h, (hidden_h, cell_h) = self.history_lstm(x_h)

        # Concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers and apply dropout
        hidden_r = torch.cat((hidden_r[-2,:,:], hidden_r[-1,:,:]), -1)
        hidden_k = torch.cat((hidden_k[-2,:,:], hidden_k[-1,:,:]), -1)
        hidden_h = torch.cat((hidden_h[-2,:,:], hidden_h[-1,:,:]), -1)
        hidden = torch.cat((hidden_r, hidden_k, hidden_h), -1)
        hidden = self.dropout(hidden)

        return self.fc(hidden)

In [271]:
RESPONSE_INPUT_DIM = len(RESPONSE.vocab)
KNOWLEDGE_INPUT_DIM = len(KNOWLEDGE.vocab)
HISTORY_INPUT_DIM = len(HISTORY.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
RESPONSE_PAD_IDX = RESPONSE.vocab.stoi[RESPONSE.pad_token]
KNOWLEDGE_PAD_IDX = KNOWLEDGE.vocab.stoi[KNOWLEDGE.pad_token]
HISTORY_PAD_IDX = HISTORY.vocab.stoi[HISTORY.pad_token]

model = LSTM(RESPONSE_INPUT_DIM,
             KNOWLEDGE_INPUT_DIM,
             HISTORY_INPUT_DIM,
             EMBEDDING_DIM,
             HIDDEN_DIM,
             OUTPUT_DIM,
             N_LAYERS,
             BIDIRECTIONAL,
             DROPOUT,
             RESPONSE_PAD_IDX,
             KNOWLEDGE_PAD_IDX,
             HISTORY_PAD_IDX)

In [272]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 30,523,669 trainable parameters


In [273]:
print(RESPONSE.vocab.vectors.shape)
print(KNOWLEDGE.vocab.vectors.shape)
print(HISTORY.vocab.vectors.shape)

torch.Size([25002, 300])
torch.Size([24539, 300])
torch.Size([25002, 300])


In [274]:
model.response_embedding.weight.data.copy_(RESPONSE.vocab.vectors)
model.knowledge_embedding.weight.data.copy_(KNOWLEDGE.vocab.vectors)
model.history_embedding.weight.data.copy_(HISTORY.vocab.vectors)

tensor([[ 0.5307, -0.1979,  0.3649,  ..., -0.2608, -0.4691, -0.4907],
        [ 0.4916, -0.3828, -1.8239,  ..., -1.8526,  0.6029, -0.6282],
        [ 0.2013,  0.0104,  0.1623,  ..., -0.0931, -0.1408, -0.1326],
        ...,
        [-1.0796,  0.2816,  0.0254,  ..., -0.0728, -0.7381,  1.4073],
        [ 0.5581, -0.1012,  0.1553,  ..., -1.9899, -1.2013,  0.5487],
        [-0.6868, -0.2798, -0.1581,  ...,  1.1993, -2.3149,  0.6361]])

In [276]:
UNK_IDX_R = RESPONSE.vocab.stoi[RESPONSE.unk_token]
UNK_IDX_K = RESPONSE.vocab.stoi[KNOWLEDGE.unk_token]
UNK_IDX_H = RESPONSE.vocab.stoi[HISTORY.unk_token]

model.response_embedding.weight.data[UNK_IDX_R] = torch.zeros(EMBEDDING_DIM)
model.response_embedding.weight.data[RESPONSE_PAD_IDX] = torch.zeros(EMBEDDING_DIM)

model.knowledge_embedding.weight.data[UNK_IDX_K] = torch.zeros(EMBEDDING_DIM)
model.knowledge_embedding.weight.data[KNOWLEDGE_PAD_IDX] = torch.zeros(EMBEDDING_DIM)

model.history_embedding.weight.data[UNK_IDX_H] = torch.zeros(EMBEDDING_DIM)
model.history_embedding.weight.data[HISTORY_PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.response_embedding.weight.data)
print(model.knowledge_embedding.weight.data)
print(model.history_embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0569, -0.0520,  0.2733,  ..., -0.0695, -0.1606, -0.0989],
        ...,
        [ 0.7385,  0.2614,  0.3067,  ..., -0.1981, -0.2725, -0.0737],
        [ 0.3933, -0.1404, -0.0947,  ...,  0.0495,  0.0273, -0.0339],
        [ 0.1373, -0.1097,  0.1443,  ...,  0.0776, -0.1282, -0.0274]])
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.2013,  0.0104,  0.1623,  ..., -0.0931, -0.1408, -0.1326],
        ...,
        [-1.9236,  1.6207,  0.0922,  ...,  0.4093, -0.4299, -1.4215],
        [ 0.0026, -1.8441, -0.8412,  ...,  0.9285,  0.1814,  0.7505],
        [ 0.9415, -1.7080,  0.7130,  ..., -0.9283,  0.6269,  1.0827]])
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0

In [277]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [278]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [279]:
from sklearn.metrics import f1_score


def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc


def binary_f1(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    f1 = f1_score(rounded_preds.detach().cpu(), y.detach().cpu(), average="macro")

    return f1


In [280]:
def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:

        optimizer.zero_grad()

        response, response_lengths = batch.r
        knowledge, knowledge_lengths = batch.k
        history, history_lengths = batch.h

        predictions = model(response, response_lengths, knowledge, knowledge_lengths, history, history_lengths).squeeze(1)

        loss = criterion(predictions, batch.l)

        acc = binary_accuracy(predictions, batch.l)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [281]:
def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_acc = 0
    epoch_f1 = 0

    model.eval()

    with torch.no_grad():

        for batch in iterator:
            response, response_lengths = batch.r
            knowledge, knowledge_lengths = batch.k
            history, history_lengths = batch.h

            predictions = model(response, response_lengths, knowledge, knowledge_lengths, history, history_lengths).squeeze(1)

            loss = criterion(predictions, batch.l)
            acc = binary_accuracy(predictions, batch.l)
            f1 = binary_f1(predictions, batch.l)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            epoch_f1 += f1.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_f1 / len(iterator)

In [282]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [283]:
N_EPOCHS = 5
# path = F"/content/gdrive/My Drive/bilstm_model.pt"
path = PROJECT_ROOT + F"/bilstm_model.pt"
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc, valid_f1 = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), path)

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% |')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% | Val. F1: {valid_f1:.3f}')

Epoch: 01 | Epoch Time: 1m 15s
	Train Loss: 0.503 | Train Acc: 74.46% |
	 Val. Loss: 0.382 |  Val. Acc: 82.95% | Val. F1: 0.785
Epoch: 02 | Epoch Time: 1m 24s
	Train Loss: 0.345 | Train Acc: 84.93% |
	 Val. Loss: 0.357 |  Val. Acc: 84.49% | Val. F1: 0.805
Epoch: 03 | Epoch Time: 1m 21s
	Train Loss: 0.296 | Train Acc: 87.66% |
	 Val. Loss: 0.348 |  Val. Acc: 85.18% | Val. F1: 0.816
Epoch: 04 | Epoch Time: 1m 21s
	Train Loss: 0.252 | Train Acc: 89.70% |
	 Val. Loss: 0.366 |  Val. Acc: 84.39% | Val. F1: 0.808
Epoch: 05 | Epoch Time: 1m 21s
	Train Loss: 0.214 | Train Acc: 91.27% |
	 Val. Loss: 0.375 |  Val. Acc: 85.08% | Val. F1: 0.813


In [284]:
model.load_state_dict(torch.load(path, map_location=device))

test_loss, test_acc, tesr_f1 = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% | Test F1: {valid_f1:.2f}')

Test Loss: 0.346 | Test Acc: 85.22% | Test F1: 0.81


In [285]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_hallucination(model, knowledge, response):
    model.eval()

    tokenized_r = [tok.text for tok in nlp.tokenizer(response)]
    indexed_r = [RESPONSE.vocab.stoi[t] for t in tokenized_r]
    length_r = [len(indexed_r)]
    tensor_r = torch.LongTensor(indexed_r).to(device)
    tensor_r = tensor_r.unsqueeze(1)
    length_tensor_r = torch.LongTensor(length_r)

    tokenized_k = [tok.text for tok in nlp.tokenizer(knowledge)]
    indexed_k = [KNOWLEDGE.vocab.stoi[t] for t in tokenized_k]
    length_k = [len(indexed_k)]
    tensor_k = torch.LongTensor(indexed_k).to(device)
    tensor_k = tensor_k.unsqueeze(1)
    length_tensor_k = torch.LongTensor(length_k)

    prediction = torch.sigmoid(model(tensor_r, length_tensor_r, tensor_k, length_tensor_k))

    return prediction.item()


In [286]:
predict_hallucination(model, "", "I love dogs")

TypeError: ignored

In [ ]:
predict_hallucination(model, "", "Dogs are animals.")

In [ ]:
predict_hallucination(model, "", "I was walking my dog last week.")

In [ ]:
predict_hallucination(model, "", "Dogs need to be walked daily.")

In [ ]:
test_data[2].r

In [ ]:
predict_hallucination(model, "", "Dylan's Candy Bar is a candy supplier.")

In [ ]:
predict_hallucination(model, "", "Dylan's Candy Bar is my favorite great brand of candy.")

In [289]:
print(test_data[2].h)

['I', 'love', 'candy', ',', 'what', "'s", 'a', 'good', 'brand?\\I', 'do', "n't", 'know', 'how', 'good', 'they', 'are', ',', 'but', 'Dylan', "'s", 'Candy', 'Bar', 'has', 'a', 'chain', 'of', 'candy', 'shops', 'in', 'various', 'cities.\\Oh', ',', 'they', 'do', '?', 'What', 'kind', 'of', 'candy', 'do', 'they', 'sell', '?']
